<a href="https://colab.research.google.com/github/alexisdr/uned-tfg-deteccion-eas/blob/main/UNED-TFG-2-data-set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de datos y creación de Dataset

En este cuaderno se procesan los datos y se crea un dataset con los datos procesados y listos para el entrenamiento del modelo.

# Parámetros

* ruta_archivo_GoldStandardTrainingEA: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
* ruta_archivo_GoldStandardTestEA: equivalente al anterior pero con los datos del conjunto de pruebas.
* ruta_archivos_entrenamiento: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
* ruta_archivos_test: equivalente a training pero con el conjunto de pruebas.
* ruta_dataset: ruta en el que se almacenará el dataset
* ocurrencias_clase_none: limita las ocurrencias de la clase none para disminuir su pese en el entrenamiento. El valor 0 no realiza límite alguno.

In [51]:
ruta_base = '/drive/My Drive/CorpusPFG/'

#Datos de origen
ruta_archivo_GoldStandardTrainingEA = ruta_base + 'GoldStandardTrainingEAs.txt'
ruta_archivo_GoldStandardTestEA = ruta_base + 'GoldStandardTestEAs.txt'
ruta_archivos_entrenamiento = ruta_base + 'Training/'
ruta_archivos_test = ruta_base + 'Test/'

#Dataset procesado
ruta_dataset = ruta_base + 'Dataset'

ocurrencias_clase_none = 0

Instalación de depndencias necesarias

In [52]:
!pip install -q datasets unidecode nltk

ocurrencias_clase_noneSe realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [53]:
from google.colab import drive

drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


# Creación el conjunto de etiquetas

Se cargan los archivos para su tratamiento.

In [54]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", 
                "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(
    ruta_archivo_GoldStandardTrainingEA, sep=' ', header=None, names=column_names)
df_goldStandardTestEAs = pd.read_csv(
    ruta_archivo_GoldStandardTestEA, sep=' ', header=None, names=column_names)

In [55]:
df_goldStandardTrainingEAs

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27690409,T50.2X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24626366,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27967924,O90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25774939,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24878023,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14497,24368169,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14498,26850773,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14499,26850775,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14500,26850771,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se unen todos los códigos EAs de todas las columnas para conocer los códigos más frecuentes

In [56]:
def crear_df_cantidad(dataframe):
  ds_cantidad_TieneEA_EA1 = dataframe.TieneEA_EA1.value_counts(dropna=True, sort=True)  
  ds_cantidad_EA2 = dataframe.EA2.value_counts(dropna=True, sort=True)  
  ds_cantidad_EA3 = dataframe.EA3.value_counts(dropna=True, sort=True)
  ds_cantidad_EA4 = dataframe.EA4.value_counts(dropna=True, sort=True)
  ds_cantidad_EA5 = dataframe.EA5.value_counts(dropna=True, sort=True)
  ds_cantidad_EA6 = dataframe.EA6.value_counts(dropna=True, sort=True)
  ds_cantidad_EA7 = dataframe.EA7.value_counts(dropna=True, sort=True)
  ds_cantidad_EA8 = dataframe.EA8.value_counts(dropna=True, sort=True)
  ds_cantidad_EA9 = dataframe.EA9.value_counts(dropna=True, sort=True)

  ds_all = pd.concat([ds_cantidad_TieneEA_EA1, ds_cantidad_EA2, ds_cantidad_EA3,
                      ds_cantidad_EA4, ds_cantidad_EA5, ds_cantidad_EA6,
                      ds_cantidad_EA7, ds_cantidad_EA8, ds_cantidad_EA9])

  df_cantidad = pd.DataFrame(ds_all)
  df_cantidad = df_cantidad.reset_index()
  df_cantidad.columns = ['eas', 'counts'] 
  return df_cantidad

df_cantidad = crear_df_cantidad(df_goldStandardTrainingEAs)
df_cantidad.count()
df_cantidad

,eas,counts
0,NONE,12181
1,T38.0X5A,104
2,T45.515A,98
3,T50.2X5A,96
4,Y95,91
...,...,...
784,I95.81,1
785,T82.222A,1
786,Y84.8,1
787,T84.51XA,1


Los 8 códigos más frecuentes

In [57]:
df_cantidad.query("eas != 'NONE'").head(8)

,eas,counts
1,T38.0X5A,104
2,T45.515A,98
3,T50.2X5A,96
4,Y95,91
5,P01.1,90
6,T81.4XXA,89
7,T45.1X5A,72
8,Y83.1,71


Limita los registros con NONE a 150 muestras para rebajar su peso en el entrenamiento.

In [58]:
if (ocurrencias_clase_none > 0):
  df_none = df_goldStandardTrainingEAs.query("TieneEA_EA1 == 'NONE'").head(ocurrencias_clase_none)
  print("Datos con label none: ", df_none.count())
  df_con_label = df_goldStandardTrainingEAs.query("TieneEA_EA1 != 'NONE'")
  print("Datos sin label none: ", df_con_label.count())
  df_goldStandardTrainingEAs_none_limitados = pd.concat([df_con_label, df_none], ignore_index=True) 
  print("Datos totales: ", df_goldStandardTrainingEAs_none_limitados.count())
  df_goldStandardTrainingEAs = df_goldStandardTrainingEAs_none_limitados
  df_goldStandardTrainingEAs

Creación de las listas de etiquetas de los conjuntos de datos y su relación numérica equivalente

In [59]:
from datasets import ClassLabel

def obtener_lista_actos(dataframe, rows):
  actos = dataframe.query("Acto == 0")
  for ea in rows["eas"]:
    actos = actos.append(obtener_actos(dataframe, ea), ignore_index=True)
  return actos

def obtener_actos(dataframe, EA_code):
  query = ("TieneEA_EA1 == '{EA}'")
  query = query.replace ('{EA}', EA_code)
  return dataframe.query(query)
  
df_codigos_mas_frecuentes = df_cantidad.query("eas != 'NONE'").head(8)
print ("df_cantidad", df_cantidad)

df_entrenamiento_codigos_mas_frecuentes = obtener_lista_actos(
    df_goldStandardTrainingEAs, df_codigos_mas_frecuentes)

df_test_codigos_mas_frecuentes = obtener_lista_actos(
    df_goldStandardTestEAs, df_codigos_mas_frecuentes)


def crear_class2label (label_list):
  df = pd.DataFrame(label_list, columns=["label"])
  df = df.dropna()
  label_list = list(df.label.unique())
  return ClassLabel(num_classes=len(label_list), names=label_list)

def crear_lista_etiquetas(dataframe):
  label_list = list(dataframe.TieneEA_EA1.unique())
  label_list.extend(list(dataframe.EA2.unique()))
  label_list.extend(list(dataframe.EA3.unique()))
  label_list.extend(list(dataframe.EA4.unique()))
  label_list.extend(list(dataframe.EA5.unique()))
  label_list.extend(list(dataframe.EA6.unique()))
  label_list.extend(list(dataframe.EA7.unique()))
  label_list.extend(list(dataframe.EA8.unique()))
  label_list.extend(list(dataframe.EA9.unique()))
  label_list.extend(list(dataframe.EA10.unique()))
  return crear_class2label(label_list)

def crear_lista_etiquetas_mas_frecuentes(dataframe):
  label_list = list(dataframe.eas.unique())
  return crear_class2label(label_list)

class2label_entrenamiento = crear_lista_etiquetas(df_goldStandardTrainingEAs)
class2label_entrenamiento_masFrecuentes = crear_lista_etiquetas_mas_frecuentes(df_codigos_mas_frecuentes)
class2label_test = crear_lista_etiquetas(df_goldStandardTestEAs)
class2label_test_masFrecuentes = crear_lista_etiquetas_mas_frecuentes(df_codigos_mas_frecuentes)

df_cantidad           eas  counts
0        NONE   12181
1    T38.0X5A     104
2    T45.515A      98
3    T50.2X5A      96
4         Y95      91
..        ...     ...
784    I95.81       1
785  T82.222A       1
786     Y84.8       1
787  T84.51XA       1
788     Y84.8       1

[789 rows x 2 columns]


<ipython-input-59-a0642168836a>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actos = actos.append(obtener_actos(dataframe, ea), ignore_index=True)
<ipython-input-59-a0642168836a>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  actos = actos.append(obtener_actos(dataframe, ea), ignore_index=True)


Clases de los conjuntos de datos de entrenamiento y test

In [60]:
class2label_entrenamiento

ClassLabel(names=['T50.2X5A', 'NONE', 'O90.0', 'T46.5X5A', 'E89.0', 'H59.022', 'Y84.2', 'T83.021A', 'K94.23', 'T50.8X5A', 'Y95', 'T50.905A', 'T50.0X5A', 'L27.0', 'T85.398A', 'T38.0X5A', 'T85.79XA', 'P39.1', 'T84.498A', 'T84.82XA', 'K91.841', 'I97.618', 'T80.1XXA', 'T84.89XA', 'T40.605A', 'R50.82', 'T81.4XXA', 'T38.3X5A', 'G97.1', 'O75.2', 'N99.820', 'T45.1X5A', 'T39.1X5A', 'T85.71XA', 'L76.02', 'K66.0', 'Y83.1', 'T82.120A', 'K12.31', 'P01.1', 'G25.1', 'G89.18', 'T38.0X5D', 'L76.31', 'M96.830', 'G97.41', 'T47.4X5A', 'N99.821', 'T45.515A', 'T80.211A', 'T46.2X5S', 'P36.9', 'T36.0X5A', 'T84.84XA', 'G62.0', 'D70.1', 'T82.868A', 'T79.6XXA', 'T40.2X5A', 'T39.395A', 'T41.5X5A', 'T84.223A', 'T45.1X5D', 'T83.498A', 'L76.22', 'T43.295A', 'K94.12', 'P39.8', 'O86.29', 'K91.71', 'T84.021A', 'M96.840', 'Y83.2', 'T87.81', 'I97.130', 'T46.0X5A', 'T84.51XA', 'P03.4', 'Y64.0', 'J95.811', 'T39.2X5A', 'G97.51', 'T48.6X5A', 'T45.615A', 'H59.021', 'T82.7XXA', 'T82.330A', 'T83.29XA', 'K94.29', 'E89.2', 'E36.0

In [61]:
class2label_test

ClassLabel(names=['NONE', 'T38.0X5A', 'Y83.1', 'P02.69', 'T45.515A', 'T36.95XA', 'T50.2X5A', 'T39.315A', 'T39.95XA', 'O91.22', 'T86.10', 'Y95', 'T39.395A', 'T84.54XA', 'T84.226A', 'P01.1', 'K91.840', 'J95.811', 'T82.7XXA', 'T43.505A', 'T50.8X5A', 'E89.0', 'T45.1X5A', 'K94.29', 'T81.83XA', 'T45.615A', 'T86.5', 'K91.841', 'T88.8XXA', 'T80.1XXA', 'L76.22', 'O75.2', 'T46.0X5A', 'I97.190', 'O04.5', 'T81.4XXA', 'I95.2', 'T88.59XA', 'T36.1X5A', 'M96.1', 'T42.8X5A', 'O75.82', 'I97.121', 'T83.51XA', 'Y83.6', 'K66.0', 'T46.2X5A', 'Y84.2', 'L76.32', 'P03.89', 'K91.71', 'T84.428A', 'T85.49XA', 'T50.905A', 'G72.0', 'T83.89XA', 'T86.11', 'T42.4X5A', 'P03.4', 'K94.09', 'T80.212A', 'T39.2X5A', 'Y83.8', 'T39.4X5A', 'K12.31', 'P39.3', 'T82.09XD', 'K91.89', 'T82.9XXA', 'T82.868A', 'T42.75XA', 'G89.18', 'E89.89', 'R50.82', 'T50.3X5A', 'T40.2X5A', 'T82.524A', 'P13.4', 'T83.59XA', 'P02.7', 'T82.855D', 'T50.1X5A', 'P15.8', 'I97.51', 'T45.525A', 'N99.111', 'K91.61', 'T82.857A', 'D70.1', 'N99.512', 'P36.9', 'I

Datos de entrenamiento que están clasificados con los 8 códigos más frecuentes

In [62]:
df_entrenamiento_codigos_mas_frecuentes

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27820190,T38.0X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24640635,T38.0X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27725771,T38.0X5A,T45.1X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27834890,T38.0X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26111542,T38.0X5A,T46.5X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
706,25078241,Y83.1,T82.03XA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707,27184410,Y83.1,T82.9XXA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,26332930,Y83.1,T84.84XA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,24008892,Y83.1,T82.857D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clases de los conjuntos de datos con los códigos más frecuentes

In [63]:
class2label_entrenamiento_masFrecuentes

ClassLabel(names=['T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA', 'T45.1X5A', 'Y83.1'], id=None)

In [64]:
class2label_test_masFrecuentes

ClassLabel(names=['T38.0X5A', 'T45.515A', 'T50.2X5A', 'Y95', 'P01.1', 'T81.4XXA', 'T45.1X5A', 'Y83.1'], id=None)

# Precesamiento de datos

In [65]:
import os
import re
from unidecode import unidecode
from nltk.corpus import stopwords 
import nltk
import json
from tqdm.notebook import tqdm

re_juicio1 = re.compile(r'juicio [a-z]*')
re_juicio2 = re.compile(r'j[a-z]* clinico')
re_juicio3 = re.compile(r'j[a-z]* clinico[a-z]*')
re_juicio4 = re.compile(r'j[a-z]* diagnostico[a-z]*')
re_juicio5 = re.compile(r'j[a-z]*  clinico')
re_gestacion = re.compile(r'gestacion [a-z]*')
apartados_juicio_clinico = ["jdtco", "jc", "jd"] 
apartados_de_interes = [] #, "comentarios"
apartado_sexo = "sexo"
apartados_descartados = [
    "recomendaciones", "lopd", "pagina", "datos personales seran tratados", 
    "antecedentes personales", "tto", "tratamiento", "en cumplimiento del artículo 5", 
    "hemograma"]    

nltk.download('stopwords')

def omitir_textos_anonimizacion (linea):
  # busca expresiones del tipo _HOSPITAL_XXXX_ _DIRECCION_XXXX_
  return re.sub (r'_\w+_XXXX_', '', linea)

def omitir_no_texto (linea):
  patron = r"[\w\.]+"
  palabras = re.findall(patron, linea)
  linea = ' '.join(palabras)
  return linea# re.sub (r'[^\w]', ' ', linea)

def omitir_numeros (linea):
  return re.sub(r'[0-9]+', ' ', linea)

def es_apartado_interes (linea, incluir_sexo):
  apartados = apartados_de_interes
  if (incluir_sexo):
    apartados.append(apartado_sexo)
  for apartado in apartados:
    if (apartado in linea):
      return True
  return False

def es_apartado_gestacion (linea):
  return re_gestacion.match (linea) != None

def es_apartado_juicio_clinico (linea):
  es_apartado_jucicio_clinico = False
  for apartado in apartados_juicio_clinico:
    if (apartado in linea):
      es_apartado_jucicio_clinico = True
      break
  return (re_juicio1.match (linea) != None or
          re_juicio2.match (linea) != None or
          re_juicio3.match (linea) != None or
          re_juicio4.match (linea) != None or
          re_juicio5.match (linea) != None or 
          es_apartado_jucicio_clinico)
  
def es_apartado (linea, incluir_sexo):
  return (es_apartado_juicio_clinico(linea) or
          es_apartado_gestacion(linea) or
          es_apartado_interes (linea, incluir_sexo))

def omitir_apartados (texto):
  if (es_apartado_juicio_clinico(texto)):
    texto = re_juicio1.sub (' ', texto)
    texto = re_juicio2.sub (' ', texto)
    texto = re_juicio3.sub (' ', texto)
    texto = re_juicio4.sub (' ', texto)
    texto = re_juicio5.sub (' ', texto)    
    for apartado in apartados_juicio_clinico:
      texto = texto.replace(apartado, ' ')      
    texto = "juicio clinico: " + texto
    
  if (re_gestacion.match (texto) != None):
    texto = re_gestacion.sub (' ', texto)
    texto = "gestacion: " + texto
    
  return texto

def es_apartado_descartado (linea):
  for apartado in apartados_descartados:
    if linea.startswith(apartado):
    #if (apartado in linea):
      return True
  return False    

def omitir_stopwors (linea):
  stop_words = set(stopwords.words('spanish')) 
  palabras = linea.split() 
  resultado = ''
  for paralabra in palabras: 
      if not resultado in stop_words: 
        resultado += ' ' + paralabra
  return resultado

def normalizar_texto (linea):
  linea = unidecode(linea) 
  linea = linea.lower().strip()
  return linea

def limpiar_texto (informe_medico, diccionario_apartados, incluir_sexo):
  texto = ''
  seleccionar = False  
  apartado = ""
  texto_apartado = ""

  for lineaTexto in informe_medico:
    lineaTexto = omitir_textos_anonimizacion(lineaTexto)
    lineaTexto = normalizar_texto(lineaTexto)
    lineaTexto = omitir_no_texto(lineaTexto)
    lineaTexto = omitir_numeros(lineaTexto)
    if (not incluir_sexo and apartado_sexo in lineaTexto):
      linea_array = lineaTexto.split(" ")      
      diccionario_apartados[apartado_sexo] = linea_array[len(linea_array)-1] + "."
    elif (es_apartado(lineaTexto.strip(), incluir_sexo)):
      if (apartado != "" and apartado != lineaTexto):
        diccionario_apartados[apartado] = texto_apartado
        texto_apartado = ""
      apartado = lineaTexto   
      seleccionar = True   
    elif (es_apartado_descartado(lineaTexto.strip())):        
        if (texto_apartado != ""):
          diccionario_apartados[apartado] = texto_apartado
        apartado = "" 
        texto_apartado = ""
        seleccionar = False   
    if (seleccionar):      
      lineaTexto = omitir_stopwors(lineaTexto)  
      texto_apartado += lineaTexto 

  if (texto_apartado != ""):
    diccionario_apartados[apartado] = texto_apartado

  for apartado in diccionario_apartados.values():
    texto += apartado

  json_apartados = [ {'apartado' : k, 'texto' : diccionario_apartados[k]} for k in diccionario_apartados]
  return texto, json_apartados

def obtener_texto (acto, label_list, lista_nombres_informes_medicos, files_location, class2label):  
  texto_acto = ''  
  diccionario_apartados = {}
  for nombre_informe in lista_nombres_informes_medicos:
    with open (files_location + nombre_informe, 'rt') as informe_medico:
      resultado = limpiar_texto(informe_medico, diccionario_apartados, False)
    if (resultado[0].strip() == ""):
      with open (files_location + nombre_informe, 'rt') as informe_medico:
        resultado = limpiar_texto(informe_medico, diccionario_apartados, True)

  json_object = json.loads(json.dumps(resultado[1]))    
  for aparatado in json_object:
    texto_acto = texto_acto + omitir_apartados(aparatado['texto'].strip()) + " "

  label_list_int = convertir_codigos_EA_a_entero(class2label, label_list)
  if (len(texto_acto)<30):
    return []
  else:
    return [acto,
          label_list_int[0], 
          label_list[0], 
          label_list_int,
          label_list, 
          lista_nombres_informes_medicos, 
          texto_acto, 
          json.dumps(resultado[1])]

def convertir_codigos_EA_a_entero(class2label, label_list):
  label_list_int = list()
  for codigo_ea in label_list:
    try:
      label_list_int.append(class2label.str2int(codigo_ea))
    except ValueError:
      pass
  return label_list_int

def tomar_codigos_EA_de_todas_las_columnas(row):
  label_list = list()
  for col in column_names:
    if (col == column_names[0]):
      continue
    if (type(row[col]) == str):
      label_list.append(row[col].strip())
  return label_list

def crear_vector_datos (dataframe, file_list, files_location, class2label, cantidad): 
  ds_resultado = []
  i = 0
  for index, row in tqdm(dataframe.iterrows()):
    texto_filtro = str(row['Acto']) + '-'
    archivos_acto = filter(lambda x: texto_filtro in x, file_list)       
    datos = obtener_texto(row['Acto'], tomar_codigos_EA_de_todas_las_columnas(row), 
                                      list(archivos_acto), files_location, class2label)
    if (datos):
      ds_resultado.append(datos)
    i += 1
    if (cantidad > 0 and i > cantidad):
     break
  return ds_resultado

def crear_dataframe (labeled_array):
  return pd.DataFrame(labeled_array, columns=[
      'acto', 'label', 'label_str', 
      'label_list', 'label_list_str', 
      'informes', 'text', 'json'])

def crear_dataframe_para_un_informe (nombre_archivo_informe):
  acto = nombre_archivo_informe.split("-")[0]
  labeled_array_training = crear_vector_datos (
      df_goldStandardTrainingEAs.query("Acto == " + acto), 
       [nombre_archivo_informe], ruta_archivos_entrenamiento, 
       class2label_entrenamiento, 0)
  return crear_dataframe (labeled_array_training)  

def crear_dataframe_train_test ():
  lista_archivos_entrenamiento = [f for f in os.listdir(ruta_archivos_entrenamiento) 
    if os.path.isfile(os.path.join(ruta_archivos_entrenamiento, f))]

  vector_datos_entrenamiento_masFrecuentes = crear_vector_datos (
      df_entrenamiento_codigos_mas_frecuentes, lista_archivos_entrenamiento, 
      ruta_archivos_entrenamiento, class2label_entrenamiento_masFrecuentes, 0)
  df_entrenamiento_masFrecuentes = crear_dataframe (vector_datos_entrenamiento_masFrecuentes)

  vector_datos_entrenamiento = crear_vector_datos (
      df_goldStandardTrainingEAs, lista_archivos_entrenamiento, 
      ruta_archivos_entrenamiento, class2label_entrenamiento, 0)
  df_entrenamiento = crear_dataframe (vector_datos_entrenamiento)  
  
  lista_archivos_test = [f for f in os.listdir(ruta_archivos_test) 
    if os.path.isfile(os.path.join(ruta_archivos_test, f))]
  
  vector_datos_test = crear_vector_datos (
      df_goldStandardTestEAs, lista_archivos_test, ruta_archivos_test, class2label_test, 0)
  df_test = crear_dataframe (vector_datos_test)
  
  vector_datos_test_mas_frecuentes = crear_vector_datos (
      df_test_codigos_mas_frecuentes, lista_archivos_test, ruta_archivos_test, class2label_test_masFrecuentes, 0)
  df_test_masFrecuentes = crear_dataframe (vector_datos_test_mas_frecuentes)

  return df_entrenamiento, df_test, df_entrenamiento_masFrecuentes, df_test_masFrecuentes

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [66]:
crear_dataframe_para_un_informe("26254234-167498204.txt")

0it [00:00, ?it/s]

,acto,label,label_str,label_list,label_list_str,informes,text,json
0,26254234,15,T38.0X5A,[15],[T38.0X5A],[26254234-167498204.txt],mujer. juicio clinico: panendoscopia oral no...,"[{""apartado"": ""sexo"", ""texto"": ""mujer.""}, {""ap..."


In [67]:
dataframes = crear_dataframe_train_test()

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [68]:
dataframes[0]

,acto,label,label_str,label_list,label_list_str,informes,text,json
0,27690409,0,T50.2X5A,[0],[T50.2X5A],[27690409-171690541.txt],hombre. juicio clinico: primer episodio de d...,"[{""apartado"": ""sexo"", ""texto"": ""hombre.""}, {""a..."
1,24626366,1,NONE,[1],[NONE],[24626366-163054389.txt],mujer. juicio clinico: neumonia probablement...,"[{""apartado"": ""sexo"", ""texto"": ""mujer.""}, {""ap..."
2,27967924,2,O90.0,[2],[O90.0],[27967924-172443607.txt],gestacion: f.u.r f.p.p. . gestacion: la ...,"[{""apartado"": ""gestacion actual"", ""texto"": "" g..."
3,25774939,1,NONE,[1],[NONE],[25774939-166138278.txt],hombre. juicio clinico: episodio de icc leve...,"[{""apartado"": ""sexo"", ""texto"": ""hombre.""}, {""a..."
4,24878023,1,NONE,[1],[NONE],[24878023-163476964.txt],hombre. juicio clinico: celulitis perirobita...,"[{""apartado"": ""sexo"", ""texto"": ""hombre.""}, {""a..."
...,...,...,...,...,...,...,...,...
14333,25849325,1,NONE,[1],[NONE],[25849325-166352919.txt],mujer. juicio clinico: aborto diferido de se...,"[{""apartado"": ""sexo"", ""texto"": ""mujer.""}, {""ap..."
14334,24368169,1,NONE,[1],[NONE],[24368169-162102373.txt],mujer. juicio clinico: anemizacion aguda sob...,"[{""apartado"": ""sexo"", ""texto"": ""mujer.""}, {""ap..."
14335,26850773,1,NONE,[1],[NONE],"[26850773-169269452.txt, 26850773-169270850.txt]",juicio clinico: cancer de mama derecha. ante...,"[{""apartado"": ""juicio clinico"", ""texto"": "" jui..."
14336,26850775,1,NONE,[1],[NONE],[26850775-169239813.txt],mujer. juicio clinico: intolerancia a aines ...,"[{""apartado"": ""sexo"", ""texto"": ""mujer.""}, {""ap..."


# Creación del Dataset

In [69]:
from datasets import Dataset, Value, DatasetDict

def set_dataset_features (dataset, class2label):
  new_features = dataset.features.copy()
  new_features["label"] = class2label
  return dataset.cast(new_features)
  
TRAIN = 0
TEST = 1
TRAIN_MAS_FRECUENTES = 2
TEST_MAS_FRECUENTES = 3

dataset_entrenamiento = Dataset.from_pandas(dataframes[TRAIN])
dataset_entrenamiento = set_dataset_features (dataset_entrenamiento, class2label_entrenamiento)
dataset_entrenamiento_validacion = dataset_entrenamiento.train_test_split(test_size=0.1)

dataset_test = Dataset.from_pandas(dataframes[TEST])
dataset_test = set_dataset_features (dataset_test, class2label_test)

dataset_entrenamiento_masFrecuentes = Dataset.from_pandas(dataframes[TRAIN_MAS_FRECUENTES])
dataset_entrenamiento_masFrecuentes = set_dataset_features (dataset_entrenamiento_masFrecuentes, class2label_entrenamiento_masFrecuentes)
dataset_entrenamiento_validacion_masFrecuentes = dataset_entrenamiento_masFrecuentes.train_test_split(test_size=0.1)

dataset_test_masFrecuentes = Dataset.from_pandas(dataframes[TEST_MAS_FRECUENTES])
dataset_test_masFrecuentes = set_dataset_features (dataset_test_masFrecuentes, class2label_test_masFrecuentes)

dataset_completo = DatasetDict(
    {
        "train": dataset_entrenamiento_validacion["train"],
        "validation": dataset_entrenamiento_validacion["test"],
        "test": dataset_test,
        "trainMasFrecuentes": dataset_entrenamiento_validacion_masFrecuentes["train"],
        "validationMasFrecuentes": dataset_entrenamiento_validacion_masFrecuentes["test"],
        "testMasFrecuentes": dataset_test_masFrecuentes
    })

dataset_completo

Casting the dataset:   0%|          | 0/14338 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3572 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/708 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/175 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 12904
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 1434
    })
    test: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 3572
    })
    trainMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 637
    })
    validationMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
        num_rows: 71
    })
    testMasFrecuentes: Dataset({
        features: ['acto', 'label', 'label_str', 'label_list', 'label_list_str', 'informes', 'text', 'json'],
     

In [70]:
dataset_completo["train"][0]

{'acto': 27069272,
 'label': 1,
 'label_str': 'NONE',
 'label_list': [1],
 'label_list_str': ['NONE'],
 'informes': ['27069272-169841393.txt',
  '27069272-169863753.txt',
  '27069272-169876955.txt'],
 'text': 'hombre. juicio clinico:   infarto agudo de miocardio de localizacion posterolateral killip i. trombosis tardia del stent proximal de bisectriz arteria responsable . tromboaspiracion y actp con balon x mm no compliante guiado por ivus sobre stent antiguo de bisectriz proximal. excelente resultado angiografico y por ecografia intracoronaria. stent en ramo bisectriz distal y cd media permeables. reestenosis oclusiva de stents previamente implantados en da media y distal escaso vaso distal residual . hipertrofia ventricular izquierda leve con funcion ventricular conservada. aquinesia media lateral posterior y segmentos medio y basal posterior. bronconeumopatia cronica. nodulo pulmonar solitario en lsi al parecer en su pais hace meses se realizo una radiografia de torax con hallazgos 

In [71]:
dataset_completo["train"][0]['text']

'hombre. juicio clinico:   infarto agudo de miocardio de localizacion posterolateral killip i. trombosis tardia del stent proximal de bisectriz arteria responsable . tromboaspiracion y actp con balon x mm no compliante guiado por ivus sobre stent antiguo de bisectriz proximal. excelente resultado angiografico y por ecografia intracoronaria. stent en ramo bisectriz distal y cd media permeables. reestenosis oclusiva de stents previamente implantados en da media y distal escaso vaso distal residual . hipertrofia ventricular izquierda leve con funcion ventricular conservada. aquinesia media lateral posterior y segmentos medio y basal posterior. bronconeumopatia cronica. nodulo pulmonar solitario en lsi al parecer en su pais hace meses se realizo una radiografia de torax con hallazgos patologicos de lo que no aporta informes . hta. dislipemia con control suboptimo. glucemia basal levemente elevada sin criterios de diabetes mellitus. cardiopatia isquemica cronica enfermedad de vasos revascul

In [72]:
dataset_completo["train"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['T50.2X5A', 'NONE', 'O90.0', 'T46.5X5A', 'E89.0', 'H59.022', 'Y84.2', 'T83.021A', 'K94.23', 'T50.8X5A', 'Y95', 'T50.905A', 'T50.0X5A', 'L27.0', 'T85.398A', 'T38.0X5A', 'T85.79XA', 'P39.1', 'T84.498A', 'T84.82XA', 'K91.841', 'I97.618', 'T80.1XXA', 'T84.89XA', 'T40.605A', 'R50.82', 'T81.4XXA', 'T38.3X5A', 'G97.1', 'O75.2', 'N99.820', 'T45.1X5A', 'T39.1X5A', 'T85.71XA', 'L76.02', 'K66.0', 'Y83.1', 'T82.120A', 'K12.31', 'P01.1', 'G25.1', 'G89.18', 'T38.0X5D', 'L76.31', 'M96.830', 'G97.41', 'T47.4X5A', 'N99.821', 'T45.515A', 'T80.211A', 'T46.2X5S', 'P36.9', 'T36.0X5A', 'T84.84XA', 'G62.0', 'D70.1', 'T82.868A', 'T79.6XXA', 'T40.2X5A', 'T39.395A', 'T41.5X5A', 'T84.223A', 'T45.1X5D', 'T83.498A', 'L76.22', 'T43.295A', 'K94.12', 'P39.8', 'O86.29', 'K91.71', 'T84.021A', 'M96.840', 'Y83.2', 'T87.81', 'I97.130', 'T46.0X5A', 'T84.51XA', 'P03.4', 'Y64.0', 'J95.811', 'T39.2X5A', 'G97.51', 'T48.6X5A', 'T45.615A', 'H59.021', 'T82.7XXA',

In [76]:
dataset_completo["test"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['NONE', 'T38.0X5A', 'Y83.1', 'P02.69', 'T45.515A', 'T36.95XA', 'T50.2X5A', 'T39.315A', 'T39.95XA', 'O91.22', 'T86.10', 'Y95', 'T39.395A', 'T84.54XA', 'T84.226A', 'P01.1', 'K91.840', 'J95.811', 'T82.7XXA', 'T43.505A', 'T50.8X5A', 'E89.0', 'T45.1X5A', 'K94.29', 'T81.83XA', 'T45.615A', 'T86.5', 'K91.841', 'T88.8XXA', 'T80.1XXA', 'L76.22', 'O75.2', 'T46.0X5A', 'I97.190', 'O04.5', 'T81.4XXA', 'I95.2', 'T88.59XA', 'T36.1X5A', 'M96.1', 'T42.8X5A', 'O75.82', 'I97.121', 'T83.51XA', 'Y83.6', 'K66.0', 'T46.2X5A', 'Y84.2', 'L76.32', 'P03.89', 'K91.71', 'T84.428A', 'T85.49XA', 'T50.905A', 'G72.0', 'T83.89XA', 'T86.11', 'T42.4X5A', 'P03.4', 'K94.09', 'T80.212A', 'T39.2X5A', 'Y83.8', 'T39.4X5A', 'K12.31', 'P39.3', 'T82.09XD', 'K91.89', 'T82.9XXA', 'T82.868A', 'T42.75XA', 'G89.18', 'E89.89', 'R50.82', 'T50.3X5A', 'T40.2X5A', 'T82.524A', 'P13.4', 'T83.59XA', 'P02.7', 'T82.855D', 'T50.1X5A', 'P15.8', 'I97.51', 'T45.525A', 'N99.111', 'K9

In [77]:
dataset_completo.save_to_disk(ruta_dataset)

Saving the dataset (0/1 shards):   0%|          | 0/12904 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1434 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3572 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/637 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/71 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/175 [00:00<?, ? examples/s]